In [101]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostRegressor
import lightgbm as lgb
import numpy as np

In [102]:
df=pd.read_excel('D:\\Data_Train.xlsx')
df.head(6)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300
5,CASUAL DINING,5618,"North Indian, Chinese, Seafood, Biryani",12noon – 12:30AM (Mon-Sun),Mumbai,Vile Parle West,4.0,550 votes,800


In [103]:
en=OneHotEncoder()
df.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [104]:
df['LOCALITY'].nunique()

1416

In [105]:
df['CITY']=df['CITY'].str.split().str.len()
df=df.drop(['TITLE','LOCALITY'],axis=1)
df['CUISINES']=df['CUISINES'].str.split().str.len()
a=df['TIME'].str.split(',')
df['LUNCH']=a.str[0]
df['DINNER']=a.str[1]
df['TIME'].head()

0    11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)
1                      6pm – 11pm (Mon-Sun)
2       11am – 3:30pm, 7pm – 11pm (Mon-Sun)
3                   11:30am – 1am (Mon-Sun)
4                      11am – 1am (Mon-Sun)
Name: TIME, dtype: object

In [106]:
df.head(6)

,RESTAURANT_ID,CUISINES,TIME,CITY,RATING,VOTES,COST,LUNCH,DINNER
0,9438,4,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",1.0,3.6,49 votes,1200,11am – 4pm,7:30pm – 11:30pm (Mon-Sun)
1,13198,4,6pm – 11pm (Mon-Sun),1.0,4.2,30 votes,1500,6pm – 11pm (Mon-Sun),NaN
2,10915,5,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",1.0,3.8,221 votes,800,11am – 3:30pm,7pm – 11pm (Mon-Sun)
3,6346,2,11:30am – 1am (Mon-Sun),1.0,4.1,24 votes,800,11:30am – 1am (Mon-Sun),NaN
4,15387,1,11am – 1am (Mon-Sun),1.0,3.8,165 votes,300,11am – 1am (Mon-Sun),NaN
5,5618,5,12noon – 12:30AM (Mon-Sun),1.0,4.0,550 votes,800,12noon – 12:30AM (Mon-Sun),NaN


In [107]:
#df[df['LUNCH'].notnull()]=1

In [108]:
df['DINNER']=df['DINNER'].fillna(0)
df.head()

,RESTAURANT_ID,CUISINES,TIME,CITY,RATING,VOTES,COST,LUNCH,DINNER
0,9438,4,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",1.0,3.6,49 votes,1200,11am – 4pm,7:30pm – 11:30pm (Mon-Sun)
1,13198,4,6pm – 11pm (Mon-Sun),1.0,4.2,30 votes,1500,6pm – 11pm (Mon-Sun),0
2,10915,5,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",1.0,3.8,221 votes,800,11am – 3:30pm,7pm – 11pm (Mon-Sun)
3,6346,2,11:30am – 1am (Mon-Sun),1.0,4.1,24 votes,800,11:30am – 1am (Mon-Sun),0
4,15387,1,11am – 1am (Mon-Sun),1.0,3.8,165 votes,300,11am – 1am (Mon-Sun),0


In [109]:
#df['DINNER_FROM']=df['DINNER_FROM'].fillna(0)
#df['DINNER_TO']=df['DINNER_TO'].fillna(0)
df['DINNER'].head(15)

0      7:30pm – 11:30pm (Mon-Sun)
1                               0
2            7pm – 11pm (Mon-Sun)
3                               0
4                               0
5                               0
6                             Tue
7                               0
8                               0
9         4pm – 10:30pm (Mon-Sun)
10                              0
11                            Tue
12                              0
13                              0
14          11am – 11pm (Tue-Sun)
Name: DINNER, dtype: object

In [110]:
df['DINNER_FROM']=df['DINNER'].str.extract('(\d+)')
df=df.drop('DINNER',axis=1)
df['DINNER_FROM']=df['DINNER_FROM'].fillna(0)
df['LUNCH_FROM']=df['LUNCH'].str.extract('(\d+)')
df['LUNCH_FROM']=df['LUNCH_FROM'].fillna(0)

In [111]:
df.isnull().sum()

RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
RATING              2
VOTES            1204
COST                0
LUNCH               0
DINNER_FROM         0
LUNCH_FROM          0
dtype: int64

In [112]:
df['CITY']=df['CITY'].fillna(1.0)
df['VOTES']=df['VOTES'].fillna('44 votes')
a=df['VOTES'].str.split()
df['VOTES']=a.str[0]
df=df.drop(['TIME','LUNCH'],axis=1)

In [113]:
df['RATING']=df["RATING"].str.replace('NEW','0')
df['RATING']=df["RATING"].str.replace('-','0')
df['RATING']=df['RATING'].astype('float')
df['VOTES']=df['VOTES'].astype('float')

In [114]:
df['DINNER_FROM']=df['DINNER_FROM'].astype('float')
df['LUNCH_FROM']=df['LUNCH_FROM'].astype('float')
df['RATING']=df['RATING'].fillna(3.9)

In [115]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale

In [116]:
x=df.drop(['COST','RESTAURANT_ID'],axis=1)
y=df['COST']

In [117]:
cat=CatBoostRegressor(iterations=120, depth=4, learning_rate=0.16, loss_function='RMSE',
                     )
cat.fit(x_train,y_train)

0:	learn: 831.2240396	total: 37.5ms	remaining: 4.46s
1:	learn: 767.8629867	total: 67ms	remaining: 3.95s
2:	learn: 715.4119488	total: 101ms	remaining: 3.95s
3:	learn: 675.3031894	total: 136ms	remaining: 3.93s
4:	learn: 643.8083490	total: 176ms	remaining: 4.04s
5:	learn: 619.1459732	total: 208ms	remaining: 3.95s
6:	learn: 599.9903005	total: 249ms	remaining: 4.02s
7:	learn: 584.9738354	total: 297ms	remaining: 4.15s
8:	learn: 575.1652140	total: 341ms	remaining: 4.21s
9:	learn: 568.3976791	total: 383ms	remaining: 4.21s
10:	learn: 562.4941273	total: 417ms	remaining: 4.13s
11:	learn: 555.9028121	total: 452ms	remaining: 4.07s
12:	learn: 552.6488369	total: 488ms	remaining: 4.01s
13:	learn: 549.7752413	total: 534ms	remaining: 4.04s
14:	learn: 545.9533423	total: 575ms	remaining: 4.02s
15:	learn: 543.7205167	total: 613ms	remaining: 3.98s
16:	learn: 542.5852978	total: 656ms	remaining: 3.97s
17:	learn: 541.3277420	total: 717ms	remaining: 4.06s
18:	learn: 540.1997098	total: 764ms	remaining: 4.06s
19:

In [118]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
lasso=Lasso(alpha=0.01)
lasso.fit(x_train,y_train)
y_lasso=lasso.predict(x_test)
np.sqrt(mean_squared_error(y_lasso,y_test))

549.1675947642777

In [119]:
test=pd.read_excel('D:\\Data_Test.xlsx')
test.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


In [120]:
test['CITY']=test['CITY'].str.split().str.len()
test=test.drop(['TITLE','LOCALITY'],axis=1)
test['CUISINES']=test['CUISINES'].str.split().str.len()
a=test['TIME'].str.split(',')
test['LUNCH']=a.str[0]
test['DINNER']=a.str[1]

In [121]:
test['DINNER']=test['DINNER'].fillna(0)

In [122]:
test['DINNER_FROM']=test['DINNER'].str.extract('(\d+)')
test=test.drop('DINNER',axis=1)
test['DINNER_FROM']=test['DINNER_FROM'].fillna(0)
test['LUNCH_FROM']=test['LUNCH'].str.extract('(\d+)')
test['LUNCH_FROM']=test['LUNCH_FROM'].fillna(0)

In [123]:
test['CITY']=test['CITY'].fillna(1.0)
test['VOTES']=test['VOTES'].fillna('6 votes')
a=test['VOTES'].str.split()
test['VOTES']=a.str[0]
test=test.drop(['TIME','LUNCH'],axis=1)

In [124]:
test['RATING']=test["RATING"].str.replace('NEW','0')
test['RATING']=test["RATING"].str.replace('-','0')
test['RATING']=test['RATING'].astype('float')
test['VOTES']=test['VOTES'].astype('float')

In [125]:
test['DINNER_FROM']=test['DINNER_FROM'].astype('float')
test['LUNCH_FROM']=test['LUNCH_FROM'].astype('float')
test['RATING']=test['RATING'].fillna(3.8)

In [126]:
test=test.drop('RESTAURANT_ID',axis=1)

In [127]:
y_test=cat.predict(test)

In [128]:
y_test=y_test.astype('int')

In [129]:
final=pd.DataFrame(y_test)
final.columns=['COST']
final.to_excel('F:\\submission.xlsx',index=False)

In [130]:
final

,COST
0,1272
1,957
2,625
3,594
4,402
5,350
6,602
7,1214
8,669
9,385
